In [2]:
import torch
from torch_geometric.data import Data
from torch_geometric.data import InMemoryDataset
import pandas as pd
import statistics
from tqdm import tqdm
import numpy as np
import random
import os

/home/huihai/anaconda3/envs/pytorch/lib/python3.7/site-packages/torch/cuda/__init__.py:83: UserWarning: CUDA initialization: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero. (Triggered internally at  /opt/conda/conda-bld/pytorch_1656352430114/work/c10/cuda/CUDAFunctions.cpp:109.)
  return torch._C._cuda_getDeviceCount() > 0


In [3]:
def get_time_frame(df, M, D, HH, MM):
    df_M = df[df['M'] == M]
    df_D = df_M[df_M['D'] == D]
    df_HH = df_D[df_D['HH'] == HH]
    df_MM = df_HH[df_HH['MM'] == MM]
    return df_MM

In [5]:
df_segments = pd.read_csv(r'../Dataset/New_Road_Segments_Divided_Dir.csv')
df_segments = df_segments[['SegmentID_', 'TrafDir','SegDir', 'ID']]

In [6]:
df_count_0 = pd.read_csv(r'../Dataset/Automated_Traffic_Volume_Counts.csv')

In [178]:
year = 2017

In [179]:
df_count = df_count_0[df_count_0['Yr'] == year]

In [1]:
Yr = df_count['Yr'].to_list()
M = df_count['M'].to_list()
D = df_count['D'].to_list()
time = []

for i in range(len(Yr)):
#     print(i)
    t = str(Yr[i]) + '_' + str(M[i]) + '_' + str(D[i])
    time.append(t)

time_list = list(set(time))
print('time len is: ' + str(len(time_list)))

NameError: name 'df_count' is not defined

In [173]:
hour_list = list(set(df_count['HH'].to_list()))
Minute_list = list(set(df_count['MM'].to_list()))

y_2019 = []
time_record = []
for t in time_list:
    print(t)
    y = [-1]*df_segments.shape[0]
    M = int(t.split('_')[1])
    D = int(t.split('_')[2])
    HH = random.choice(hour_list)
    MM = random.choice(Minute_list)
    
    time_frame = get_time_frame(df_count, M, D, HH, MM)
    
    seg_list = time_frame['SegmentID'].to_list()
    dir_list = time_frame['Direction'].to_list()
    vol_list = time_frame['Vol'].to_list()
    
    for i in range(len(seg_list)):
        seg = seg_list[i]
        direction = dir_list[i]
        vol = vol_list[i]
        
        df_row = df_segments[df_segments['SegmentID_'] == seg]
        if df_row.shape[0] == 0:
#             print('No segment: ' + str(seg))
            continue
        else:
            if df_row['TrafDir'].values[0] != 'T':
                segment_ID = df_row['ID'].values[0]
                y[segment_ID] = vol
            else:
                df_dir = df_row[df_row['SegDir'] == direction]
                if df_dir.shape[0] == 0:
#                     print("No direction: " + str(seg))
                    continue
                else:
                    segment_ID = df_dir['ID'].values[0]
                    y[segment_ID] = vol
    a = 0
    for data in y:
        if data != -1:
            a += 1
    if a > 20:
        print(a)
        y_2019.append(y) 
        time_record.append([M, D, HH, MM])

2022_10_22
2022_10_13
2022_10_19
2022_10_17
2022_10_27
2022_10_12
2022_10_16
2022_10_24
2022_10_20
2022_10_21
2022_10_14
2022_10_18
2022_10_23
2022_10_25
2022_10_26
2022_10_15


In [168]:
y_2019_df = pd.DataFrame(zip(*y_2019))

path = os.path.join(r'../Data/', 'y_' + str(year) + '.csv')
path_record = os.path.join(r'../Data/', 'y_record_' + str(year) + '.csv')

y_record_df = pd.DataFrame(time_record)

print(path)
print(len(y_2019))
y_2019_df.to_csv(path)
y_record_df.to_csv(path_record)

../Data/y_2021.csv
10


In [174]:
y_2019

[]